## 1. **Setup dan Instalasi Dependencies**
- Instalasi `ultralytics` (YOLOv8) dan `supervision` untuk computer vision
- Import library yang diperlukan: OpenCV, pandas, matplotlib, seaborn, dll
- Setup warnings dan konfigurasi dasar

In [ ]:
from IPython.display import clear_output

! pip install ultralytics -qq
! pip install supervision -qq

clear_output()

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO
import supervision as sv
import subprocess
from IPython.display import Video
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json

import cv2
print(cv2.__version__)

## 2. **Konfigurasi Model dan Parameter**
- Model weights (YOLOv8s.pt)
- Parameter deteksi: confidence threshold (0.35), IOU threshold (0.5)
- Parameter heatmap: alpha, radius, kernel size
- Parameter tracking: durasi, threshold matching
- Path file video input dan output

In [ ]:
class CFG:
    ### YOLOv8/YOLOv9 custom or pretrained model
    MODEL_WEIGHTS = 'yolov8s.pt' # yolov8s.pt, yolov9c.pt, yolov9e.pt

    ### detections (YOLO)
    CONFIDENCE = 0.35
    IOU = 0.5

    ### heatmap (Supervision)
    HEATMAP_ALPHA = 0.30
    RADIUS = 20

    ### tracking (Supervision)
    TRACK_SECONDS = 5
    TRACK_THRESH = 0.35
    MATCH_THRESH = 0.9999

    ### paths: video file path, webcam is 0
    VIDEO_FILE = "/kaggle/input/datathon-data/part2(split-video.com).mp4"
    OUTPUT_PATH = './'

## 3. **Utilitas Video Processing**
- Fungsi `get_video_properties()` untuk mendapatkan informasi video (FPS, dimensi, durasi, dll)
- Ekstraksi dan tampilan properti video input
- Konversi codec video menggunakan FFmpeg untuk kompatibilitas

In [ ]:
def get_video_properties(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video file is opened successfully
    if not cap.isOpened():
        raise ValueError("Could not open video file")

    # Get video properties
    properties = {
        "fps": int(cap.get(cv2.CAP_PROP_FPS)),
        "frame_count": int(cap.get(cv2.CAP_PROP_FRAME_COUNT)),
        "duration_seconds": int( cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS) ),
        "width": int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
        "height": int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
        "codec": int(cap.get(cv2.CAP_PROP_FOURCC)),
    }

    # Release the video capture object
    cap.release()

    return properties

In [ ]:
video_properties = get_video_properties(CFG.VIDEO_FILE)
video_properties

In [ ]:
OUT_VIDEO_NAME = f'{CFG.OUTPUT_PATH}original_new_codec.mp4'

subprocess.run(
    [
        "ffmpeg",  "-i", CFG.VIDEO_FILE, "-crf",
        "18", "-preset", "veryfast", "-hide_banner", "-loglevel",
        "error", "-vcodec", "libx264", OUT_VIDEO_NAME
    ]
)

Video(
    data = OUT_VIDEO_NAME,
    embed = True,
    height = int(video_properties['height'] * 0.5),
    width = int(video_properties['width'] * 0.5)
)

## 4. **Inisialisasi Model dan Annotator**
- Load model YOLOv8 pretrained
- Setup heatmap annotator dengan parameter visual
- Setup label annotator untuk menampilkan ID tracker
- Konfigurasi ByteTrack untuk object tracking
- Setup video processing pipeline


In [ ]:
model = YOLO("/kaggle/working/yolov8s.pt")


In [ ]:
### heatmap config
heat_map_annotator = sv.HeatMapAnnotator(
    position = sv.Position.BOTTOM_CENTER,
    opacity = CFG.HEATMAP_ALPHA,
    radius = CFG.RADIUS,
    kernel_size = 25,
    top_hue = 0,
    low_hue = 125,
)

### annotation config
label_annotator = sv.LabelAnnotator(text_position = sv.Position.CENTER)

### tracker config
byte_tracker = sv.ByteTrack(
    track_activation_threshold=CFG.TRACK_THRESH,
    lost_track_buffer=CFG.TRACK_SECONDS * video_properties["fps"],
    minimum_matching_threshold=CFG.MATCH_THRESH,
    frame_rate=video_properties["fps"],
)

### video config
video_info = sv.VideoInfo.from_video_path(video_path = CFG.VIDEO_FILE)
frames_generator = sv.get_video_frames_generator(source_path = CFG.VIDEO_FILE, stride = 1)
output_filename = f'{CFG.OUTPUT_PATH}heatmap_output_c{int(CFG.CONFIDENCE * 100)}_iou{int(CFG.IOU * 100)}.mp4'


## 5. **Proses Deteksi dan Tracking Utama**
- Loop untuk setiap frame video
- Deteksi objek manusia (class 0) menggunakan YOLO
- Update tracker untuk konsistensi ID antar frame
- Penyimpanan tracking data (koordinat, confidence, ID, dll)
- Annotasi frame dengan heatmap dan label ID
- Export hasil ke video baru

**Output**:
- Video dengan overlay heatmap dan tracking ID
- Data tracking dalam format CSV

In [ ]:
# Tambahkan list untuk menyimpan tracking data
tracking_data = []

### Detect, track, annotate, save
with sv.VideoSink(target_path = output_filename, video_info = video_info) as sink:

    frame_idx = 0
    for frame in frames_generator:
        result = model(
            source = frame,
            classes = [0], # only person class
            conf = CFG.CONFIDENCE,
            iou = CFG.IOU,
            half = True,
            show_conf = True,
            save_txt = True,
            save_conf = True,
            save = True,
            device = 0, # dual GPU
            batch=16,
        )[0]

        detections = sv.Detections.from_ultralytics(result) # get detections

        detections = byte_tracker.update_with_detections(detections) # update tracker

        # Simpan tracking data
        for i, (bbox, tracker_id, confidence, class_id) in enumerate(zip(
            detections.xyxy,
            detections.tracker_id,
            detections.confidence,
            detections.class_id
        )):
            x1, y1, x2, y2 = bbox
            x_center = (x1 + x2) / 2 / frame.shape[1]  # normalize
            y_center = (y1 + y2) / 2 / frame.shape[0]  # normalize
            width = (x2 - x1) / frame.shape[1]  # normalize
            height = (y2 - y1) / frame.shape[0]  # normalize

            tracking_data.append({
                'frame': frame_idx,
                'tracker_id': tracker_id,
                'class': class_id,
                'class_name': model.names[class_id],
                'x_center': x_center,
                'y_center': y_center,
                'width': width,
                'height': height,
                'confidence': confidence
            })

        ### draw heatmap
        annotated_frame = heat_map_annotator.annotate(
            scene = frame.copy(),
            detections = detections
        )

        ### draw other attributes from `detections` object
        labels = [
            f"#{tracker_id}"
            for class_id, tracker_id
            in zip(detections.class_id, detections.tracker_id)
        ]

        label_annotator.annotate(
            scene = annotated_frame,
            detections = detections,
            labels = labels
        )

        sink.write_frame(frame = annotated_frame)
        frame_idx += 1

clear_output()


In [ ]:
# Simpan tracking data ke CSV
tracking_df = pd.DataFrame(tracking_data)
tracking_df.to_csv(f'{CFG.OUTPUT_PATH}tracking_results.csv', index=False)

## 6. **Konversi dan Tampilan Hasil**
- Konversi video hasil dengan codec yang kompatibel
- Tampilan video hasil dalam notebook
- Parsing dan tampilan data prediksi dari file YOLO format

In [ ]:
IN_VIDEO_NAME = f'{CFG.OUTPUT_PATH}heatmap_output_c{int(CFG.CONFIDENCE * 100)}_iou{int(CFG.IOU * 100)}.mp4' # detected video on output or predict folder
OUT_VIDEO_NAME = f'{CFG.OUTPUT_PATH}heatmap_and_track.mp4' # detected video new codec

subprocess.run(
    [
        "ffmpeg",  "-i", IN_VIDEO_NAME, "-crf",
        "18", "-preset", "veryfast", "-hide_banner", "-loglevel",
        "error", "-vcodec", "libx264", OUT_VIDEO_NAME
    ]
)

Video(
    data = OUT_VIDEO_NAME,
    embed = True,
    height = int(video_properties['height'] * 0.5),
    width = int(video_properties['width'] * 0.5)
)

In [ ]:
file_path = f'{CFG.OUTPUT_PATH}runs/detect/predict/labels/image0.txt'

columns = ['class', 'x_center', 'y_center', 'width', 'height', 'confidence']
predictions = pd.read_csv(file_path, delimiter=' ', header=None, names=columns)

### add column class_name
predictions['class_name'] = predictions['class'].map(model.names)
predictions = predictions[['class_name'] + columns] # reorder
predictions

## 7. **Fungsi Analisis Data Tracking**
- `generate_person_logs_and_heatmap()`: Konversi tracking data ke format analisis
- `generate_person_log()`: Membuat log detail untuk setiap person
- `generate_heatmap_data()`: Membuat data heatmap berdasarkan grid
- `generate_summary_stats()`: Statistik ringkasan keseluruhan
- Konversi koordinat normalized ke pixel coordinates

In [ ]:
def generate_person_logs_and_heatmap(tracking_data, video_properties, output_path='./'):
    """
    Generate person detection logs and heatmap data from existing tracking results.

    Args:
        tracking_data (list): Your existing tracking_data list
        video_properties (dict): Your existing video_properties dict
        output_path (str): Path to save output files (default: './')

    Returns:
        tuple: (person_log_df, heatmap_data_df, summary_stats)
    """

    print("🔄 Processing tracking data for person logs and heatmap...")

    # Convert tracking data to DataFrame
    df = pd.DataFrame(tracking_data)

    # Calculate time stamps
    fps = video_properties['fps']
    df['timestamp_seconds'] = df['frame'] / fps
    df['timestamp_formatted'] = df['timestamp_seconds'].apply(
        lambda x: str(timedelta(seconds=int(x)))
    )

    # Convert normalized coordinates back to pixel coordinates
    frame_width = video_properties['width']
    frame_height = video_properties['height']

    df['x_pixel'] = df['x_center'] * frame_width
    df['y_pixel'] = df['y_center'] * frame_height
    df['bbox_left'] = (df['x_center'] - df['width']/2) * frame_width
    df['bbox_top'] = (df['y_center'] - df['height']/2) * frame_height
    df['bbox_right'] = (df['x_center'] + df['width']/2) * frame_width
    df['bbox_bottom'] = (df['y_center'] + df['height']/2) * frame_height

    print(f"📊 Found {df['tracker_id'].nunique()} unique persons in {len(df)} total detections")

    return df, fps, frame_width, frame_height

In [ ]:
def generate_person_log(df, fps, frame_width, frame_height):
    """Generate detailed person detection logs"""

    person_log = []

    for person_id in df['tracker_id'].unique():
        person_data = df[df['tracker_id'] == person_id].sort_values('frame')

        # Calculate person statistics
        first_detection = person_data.iloc[0]
        last_detection = person_data.iloc[-1]
        total_detections = len(person_data)
        avg_confidence = person_data['confidence'].mean()

        # Calculate movement statistics
        positions = person_data[['x_pixel', 'y_pixel']].values
        if len(positions) > 1:
            distances = np.sqrt(np.sum(np.diff(positions, axis=0)**2, axis=1))
            total_distance = np.sum(distances)
            max_distance = np.max(distances) if len(distances) > 0 else 0
        else:
            total_distance = 0
            max_distance = 0

        # Calculate time spent in different areas (divide frame into quadrants)
        mid_x, mid_y = frame_width / 2, frame_height / 2
        quadrant_time = {
            'top_left': 0, 'top_right': 0, 'bottom_left': 0, 'bottom_right': 0
        }

        for _, row in person_data.iterrows():
            x, y = row['x_pixel'], row['y_pixel']
            if x < mid_x and y < mid_y:
                quadrant_time['top_left'] += 1
            elif x >= mid_x and y < mid_y:
                quadrant_time['top_right'] += 1
            elif x < mid_x and y >= mid_y:
                quadrant_time['bottom_left'] += 1
            else:
                quadrant_time['bottom_right'] += 1

        # Convert frame counts to time
        for quadrant in quadrant_time:
            quadrant_time[quadrant] = quadrant_time[quadrant] / fps

        person_log.append({
            'person_id': int(person_id),
            'first_detected_frame': int(first_detection['frame']),
            'first_detected_time': first_detection['timestamp_formatted'],
            'first_detected_seconds': round(first_detection['timestamp_seconds'], 2),
            'last_detected_frame': int(last_detection['frame']),
            'last_detected_time': last_detection['timestamp_formatted'],
            'last_detected_seconds': round(last_detection['timestamp_seconds'], 2),
            'total_detection_frames': total_detections,
            'detection_duration_seconds': round(last_detection['timestamp_seconds'] - first_detection['timestamp_seconds'], 2),
            'average_confidence': round(avg_confidence, 3),
            'total_movement_distance_pixels': round(total_distance, 2),
            'max_frame_movement_pixels': round(max_distance, 2),
            'time_in_top_left_seconds': round(quadrant_time['top_left'], 2),
            'time_in_top_right_seconds': round(quadrant_time['top_right'], 2),
            'time_in_bottom_left_seconds': round(quadrant_time['bottom_left'], 2),
            'time_in_bottom_right_seconds': round(quadrant_time['bottom_right'], 2),
            'avg_x_position': round(person_data['x_pixel'].mean(), 2),
            'avg_y_position': round(person_data['y_pixel'].mean(), 2),
            'x_position_std': round(person_data['x_pixel'].std(), 2),
            'y_position_std': round(person_data['y_pixel'].std(), 2)
        })

    person_log_df = pd.DataFrame(person_log)
    print(f"👥 Generated detailed logs for {len(person_log_df)} persons")

    return person_log_df

In [ ]:
def generate_heatmap_data(df, frame_width, frame_height, fps):
    """Generate heatmap data from tracking results"""

    print("🔥 Generating heatmap data...")
    heatmap_data = []

    # Create a grid for heatmap (divide frame into cells)
    grid_size = 20  # 20x20 grid
    x_bins = np.linspace(0, frame_width, grid_size + 1)
    y_bins = np.linspace(0, frame_height, grid_size + 1)

    for i in range(grid_size):
        for j in range(grid_size):
            x_min, x_max = x_bins[i], x_bins[i + 1]
            y_min, y_max = y_bins[j], y_bins[j + 1]

            # Count detections in this grid cell
            in_cell = df[
                (df['x_pixel'] >= x_min) & (df['x_pixel'] < x_max) &
                (df['y_pixel'] >= y_min) & (df['y_pixel'] < y_max)
            ]

            detection_count = len(in_cell)
            unique_persons = in_cell['tracker_id'].nunique() if detection_count > 0 else 0
            avg_confidence = in_cell['confidence'].mean() if detection_count > 0 else 0

            heatmap_data.append({
                'grid_x': i,
                'grid_y': j,
                'x_min': round(x_min, 1),
                'x_max': round(x_max, 1),
                'y_min': round(y_min, 1),
                'y_max': round(y_max, 1),
                'x_center': round((x_min + x_max) / 2, 1),
                'y_center': round((y_min + y_max) / 2, 1),
                'detection_count': detection_count,
                'unique_persons': unique_persons,
                'avg_confidence': round(avg_confidence, 3) if detection_count > 0 else 0,
                'time_spent_seconds': round(detection_count / fps, 2)
            })

    heatmap_data_df = pd.DataFrame(heatmap_data)

    return heatmap_data_df

In [ ]:
def generate_summary_stats(df, person_log_df, heatmap_data_df):
    """Generate summary statistics"""

    summary_stats = {
        'total_unique_persons': int(df['tracker_id'].nunique()),
        'total_detections': len(df),
        'video_duration_seconds': round(df['timestamp_seconds'].max(), 2),
        'average_persons_per_frame': round(df.groupby('frame')['tracker_id'].nunique().mean(), 2),
        'max_persons_in_single_frame': int(df.groupby('frame')['tracker_id'].nunique().max()),
        'most_active_person_id': int(person_log_df.loc[person_log_df['total_detection_frames'].idxmax(), 'person_id']),
        'longest_tracked_person_id': int(person_log_df.loc[person_log_df['detection_duration_seconds'].idxmax(), 'person_id']),
        'most_active_grid_cell': {
            'x': int(heatmap_data_df.loc[heatmap_data_df['detection_count'].idxmax(), 'grid_x']),
            'y': int(heatmap_data_df.loc[heatmap_data_df['detection_count'].idxmax(), 'grid_y']),
            'detections': int(heatmap_data_df['detection_count'].max())
        }
    }

    return summary_stats

## 8. **Fungsi Visualisasi**
- `plot_person_timeline()`: Timeline kehadiran setiap person
- `plot_detection_heatmap()`: Heatmap intensitas deteksi di area frame
- `plot_activity_analysis()`: Analisis aktivitas (durasi, movement, confidence)
- `display_summary_stats()`: Format tampilan statistik
- `save_results()`: Penyimpanan hasil ke file CSV dan JSON

In [ ]:
def plot_person_timeline(person_log_df, output_path, timestamp):
    """Generate person timeline visualization"""

    fig, ax = plt.subplots(figsize=(15, 8))

    colors = plt.cm.Set3(np.linspace(0, 1, len(person_log_df)))

    for i, (_, person) in enumerate(person_log_df.iterrows()):
        person_id = person['person_id']
        start_time = person['first_detected_seconds']
        end_time = person['last_detected_seconds']

        ax.barh(person_id, end_time - start_time, left=start_time, height=0.6,
                alpha=0.8, color=colors[i], edgecolor='black', linewidth=0.5)

    ax.set_xlabel('Time (seconds)', fontsize=12)
    ax.set_ylabel('Person ID', fontsize=12)
    ax.set_title('Person Detection Timeline', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f'{output_path}person_timeline_{timestamp}.png', dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()

In [ ]:
def plot_detection_heatmap(heatmap_data_df, output_path, timestamp):
    """Generate detection heatmap visualization"""

    grid_size = int(np.sqrt(len(heatmap_data_df)))
    heatmap_matrix = heatmap_data_df['detection_count'].values.reshape(grid_size, grid_size)

    fig, ax = plt.subplots(figsize=(12, 10))
    im = ax.imshow(heatmap_matrix, cmap='YlOrRd', aspect='auto')
    ax.set_title('Person Detection Heatmap\n(Warmer colors = More detections)', fontsize=14, fontweight='bold')
    ax.set_xlabel('X Grid Position (Left → Right)', fontsize=12)
    ax.set_ylabel('Y Grid Position (Top → Bottom)', fontsize=12)

    # Add colorbar
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('Detection Count', fontsize=12)

    plt.tight_layout()
    plt.savefig(f'{output_path}detection_heatmap_{timestamp}.png', dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()

In [ ]:
def plot_activity_analysis(person_log_df, output_path, timestamp):
    """Generate person activity analysis plots"""

    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

    # Detection duration distribution
    ax1.hist(person_log_df['detection_duration_seconds'], bins=min(15, len(person_log_df)),
             alpha=0.7, edgecolor='black', color='skyblue')
    ax1.set_xlabel('Detection Duration (seconds)')
    ax1.set_ylabel('Number of Persons')
    ax1.set_title('Distribution of Person Detection Durations')
    ax1.grid(True, alpha=0.3)

    # Movement distance distribution
    ax2.hist(person_log_df['total_movement_distance_pixels'], bins=min(15, len(person_log_df)),
             alpha=0.7, edgecolor='black', color='lightcoral')
    ax2.set_xlabel('Total Movement Distance (pixels)')
    ax2.set_ylabel('Number of Persons')
    ax2.set_title('Distribution of Person Movement Distances')
    ax2.grid(True, alpha=0.3)

    # Confidence distribution
    ax3.hist(person_log_df['average_confidence'], bins=min(15, len(person_log_df)),
             alpha=0.7, edgecolor='black', color='lightgreen')
    ax3.set_xlabel('Average Confidence Score')
    ax3.set_ylabel('Number of Persons')
    ax3.set_title('Distribution of Average Confidence Scores')
    ax3.grid(True, alpha=0.3)

    # Detection count per person
    ax4.bar(person_log_df['person_id'], person_log_df['total_detection_frames'],
            alpha=0.7, edgecolor='black', color='orange')
    ax4.set_xlabel('Person ID')
    ax4.set_ylabel('Total Detection Frames')
    ax4.set_title('Detection Count per Person')
    ax4.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{output_path}person_activity_analysis_{timestamp}.png', dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()

In [ ]:
def display_summary_stats(summary_stats):
    """Display summary statistics in a nice format"""
    print("\n" + "="*50)
    print("📊 DETECTION SUMMARY STATISTICS")
    print("="*50)
    print(f"👥 Total unique persons detected: {summary_stats['total_unique_persons']}")
    print(f"🎯 Total detections: {summary_stats['total_detections']:,}")
    print(f"⏱  Video duration: {summary_stats['video_duration_seconds']:.1f} seconds")
    print(f"📈 Average persons per frame: {summary_stats['average_persons_per_frame']:.2f}")
    print(f"🏆 Maximum persons in single frame: {summary_stats['max_persons_in_single_frame']}")
    print(f"🔥 Most active person: ID #{summary_stats['most_active_person_id']}")
    print(f"⏰ Longest tracked person: ID #{summary_stats['longest_tracked_person_id']}")
    print(f"📍 Hottest spot: Grid ({summary_stats['most_active_grid_cell']['x']}, {summary_stats['most_active_grid_cell']['y']}) with {summary_stats['most_active_grid_cell']['detections']} detections")
    print("="*50)

def save_results(person_log_df, heatmap_data_df, summary_stats, output_path):
    """Save all results to files"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Save person log
    person_log_filename = f'{output_path}person_detection_log_{timestamp}.csv'
    person_log_df.to_csv(person_log_filename, index=False)

    # Save heatmap data
    heatmap_filename = f'{output_path}heatmap_data_{timestamp}.csv'
    heatmap_data_df.to_csv(heatmap_filename, index=False)

    # Save summary stats
    summary_filename = f'{output_path}detection_summary_{timestamp}.json'
    with open(summary_filename, 'w') as f:
        json.dump(summary_stats, f, indent=2)

    print("\n" + "="*60)
    print("✅ ANALYSIS COMPLETE!")
    print("="*60)
    print(f"📋 Person detection log: {person_log_filename}")
    print(f"🔥 Heatmap data: {heatmap_filename}")
    print(f"📊 Summary statistics: {summary_filename}")
    print("="*60)

    return timestamp

## 9. **Eksekusi Analisis Lengkap**
- Eksekusi pipeline lengkap dari tracking data ke insights
- Generate person logs dengan detail movement dan timing
- Generate heatmap data dengan grid 20x20
- Kalkulasi summary statistics
- Export semua hasil ke file dengan timestamp

In [ ]:
# Run the complete analysis
print("🚀 Starting person detection and heatmap analysis...")

# Step 1: Process basic data
df, fps, frame_width, frame_height = generate_person_logs_and_heatmap(
    tracking_data, video_properties, CFG.OUTPUT_PATH
)

# Step 2: Generate person logs
person_log_df = generate_person_log(df, fps, frame_width, frame_height)

# Step 3: Generate heatmap data
heatmap_data_df = generate_heatmap_data(df, frame_width, frame_height, fps)

# Step 4: Generate summary statistics
summary_stats = generate_summary_stats(df, person_log_df, heatmap_data_df)

# Step 5: Save results
timestamp = save_results(person_log_df, heatmap_data_df, summary_stats, CFG.OUTPUT_PATH)

# Step 6: Display summary
display_summary_stats(summary_stats)

## 10. **Visualisasi dan Display Hasil**
- Generate dan tampilkan semua visualisasi:
  - Timeline chart person detection
  - Heatmap deteksi area
  - Histogram analisis aktivitas (durasi, movement, confidence)
- Display sample data dan top hotspots

In [ ]:
# Generate all visualizations
print("📈 Generating visualizations...")

# Timeline plot
plot_person_timeline(person_log_df, CFG.OUTPUT_PATH, timestamp)

# Heatmap plot
plot_detection_heatmap(heatmap_data_df, CFG.OUTPUT_PATH, timestamp)

# Activity analysis plots
plot_activity_analysis(person_log_df, CFG.OUTPUT_PATH, timestamp)

print("✅ All visualizations generated!")

In [ ]:
# Display first few rows of person log
print("\n👥 SAMPLE PERSON DETECTION LOG:")
print(person_log_df.head())

# Display top hotspots from heatmap
print("\n🔥 TOP 5 HOTSPOTS:")
top_hotspots = heatmap_data_df.nlargest(5, 'detection_count')[
    ['grid_x', 'grid_y', 'detection_count', 'unique_persons', 'time_spent_seconds']
]
print(top_hotspots)

## 11. **Analisis Detail Individual (Optional)**
- `analyze_specific_person()`: Analisis mendalam untuk person tertentu
- Plot trajectory movement individual
- Statistik detail movement dan behavior pattern
- Visualization path pergerakan dengan color coding berdasarkan frame

In [ ]:
# Optional: Analyze specific person in detail
def analyze_specific_person(person_id, df, person_log_df):
    """Analyze a specific person in detail"""

    # Get person data
    person_data = df[df['tracker_id'] == person_id].sort_values('frame')
    person_info = person_log_df[person_log_df['person_id'] == person_id].iloc[0]

    print(f"\n🔍 DETAILED ANALYSIS FOR PERSON ID: {person_id}")
    print("="*50)
    print(f"Detection duration: {person_info['detection_duration_seconds']:.2f} seconds")
    print(f"Total movement: {person_info['total_movement_distance_pixels']:.2f} pixels")
    print(f"Average confidence: {person_info['average_confidence']:.3f}")
    print(f"Average position: ({person_info['avg_x_position']:.1f}, {person_info['avg_y_position']:.1f})")

    # Plot person trajectory
    plt.figure(figsize=(10, 8))
    plt.plot(person_data['x_pixel'], person_data['y_pixel'], 'b-', alpha=0.7, linewidth=2)
    plt.scatter(person_data['x_pixel'], person_data['y_pixel'],
                c=person_data['frame'], cmap='viridis', s=30, alpha=0.8)
    plt.colorbar(label='Frame Number')
    plt.xlabel('X Position (pixels)')
    plt.ylabel('Y Position (pixels)')
    plt.title(f'Movement Trajectory - Person ID {person_id}')
    plt.grid(True, alpha=0.3)
    plt.gca().invert_yaxis()  # Invert Y axis to match image coordinates
    plt.show()

# Example usage (uncomment to use):
# analyze_specific_person(1, df, person_log_df)  # Analyze person with ID 1

## **Ringkasan Workflow:**

1. **Input**: Video file → **Output**: Annotated video + tracking data
2. **Processing**: YOLO detection + ByteTrack tracking + heatmap generation  
3. **Analysis**: Person logs + movement analytics + area hotspots
4. **Visualization**: Timeline charts + heatmaps + activity distributions
5. **Export**: CSV files + JSON summaries + PNG visualizations